<a href="https://colab.research.google.com/github/vanikanoria/vanikanoria/blob/main/Finetune_political_bias_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%pip install transformers datasets scikit-learn torch#
%pip install --upgrade transformers

In [ ]:
import transformers
print(transformers.__version__)

4.51.3


In [ ]:
from datasets import load_dataset
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import requests
import json
import torch
import tqdm
from itertools import chain
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bucketresearch/politicalBiasBERT")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [ ]:
#get dataset
#labels = ["Left", "Center", "Right"]
# repo = "ramybaly/Article-Bias-Prediction"
# path= "data/jsons"
# branch = "main"


In [ ]:
#!/usr/bin/env python3
"""
Pull the first N article files from the Article‑Bias‑Prediction repo,
then split each article's content into sentence‑level records.
"""

import requests, tqdm, json, re, pathlib

# ---------------------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------------------
REPO   = "ramybaly/Article-Bias-Prediction"
PATH   = "data/jsons"       # folder inside the repo that holds articles
BRANCH = "main"
N      = 10               # how many JSON files to grab (set to 10, 100, …)

# ---------------------------------------------------------------------
# 1.  LIST THE DIRECTORY
# ---------------------------------------------------------------------
api_url = f"https://api.github.com/repos/{REPO}/contents/{PATH}?ref={BRANCH}"
headers = {"Accept": "application/vnd.github.v3+json"}
files   = requests.get(api_url, headers=headers, timeout=30).json()

json_urls = [
    f["download_url"] for f in files if f["name"].endswith(".json")
][:N]

# ---------------------------------------------------------------------
# 2.  DOWNLOAD & NORMALISE
# ---------------------------------------------------------------------
all_data = []

for url in tqdm.tqdm(json_urls, desc="Downloading JSON files"):
    blob = requests.get(url, timeout=30).json()

    # The file can contain either a dict (single article) or a list
    if isinstance(blob, list):
        all_data.extend(blob)
    elif isinstance(blob, dict):
        all_data.append(blob)
    else:
        print(f"⚠️  Skipped {url}: unexpected JSON root type {type(blob)}")

print(f"Loaded {len(all_data)} original articles")

# ---------------------------------------------------------------------
# 3.  SENTENCE SPLITTER
# ---------------------------------------------------------------------
def split_sentences(text: str):
    """Yield sentences separated by '. ! ?' followed by whitespace/newline."""
    for s in re.split(r"(?<=[.!?])\s+", text.strip()):
        if s:
            yield s

# ---------------------------------------------------------------------
# 4.  BUILD SENTENCE‑LEVEL CHUNKS
# ---------------------------------------------------------------------
chunked = []

for art in all_data:
    body = art.get("content") or art.get("content_original", "")
    for idx, sent in enumerate(split_sentences(body), 1):
        rec = {**art}                        # shallow copy
        rec["content"] = rec["content_original"] = sent
        rec["ID"]      = f'{art["ID"]}_sent{idx}'
        rec["chunk_index"] = idx
        rec["is_chunk"]    = True
        chunked.append(rec)

all_data.extend(chunked)


Loaded 10 original articles


In [ ]:
training_examples = [{"text": d["content"], "label": d["bias"]} for d in all_data]

In [ ]:
dataset = Dataset.from_list(training_examples)

In [ ]:
#inspect dataset
print(dataset[0])
print(dataset.features)

{'text': 'Besides his most recent trip to Quetta , Mr. Rahami visited Karachi , Pakistan , in 2005 . Both of those cities ’ reputations have become entwined with the militant groups who have sheltered there : Karachi as a haven for the Pakistani Taliban and Al Qaeda , and Quetta as the headquarters of the exiled Afghan Taliban leadership . But both cities are also home to generations of Afghans who have fled violence in their home country .\nMuch about his New Jersey life did seem unremarkable . Amarjit Singh , a limousine driver , was friends with Mr. Rahami at Edison High School . The person he knew , he said , was a determined student with an abundance of friends and a string of girlfriends . “ Everyone seemed to like him , ” he said . “ Smart , funny , humble . ”\nHe viewed the teenage Mr. Rahami as the prototypical immigrant , teetering between two worlds . While he wore jeans and sweatshirts like his friends and worked at a Pathmark supermarket after school , he preferred Afghan 

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
#tokenize dataset
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_ds = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/368 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./politicalbiasbert-finetuned",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    save_steps=500,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # 👈 here!
)

trainer.train()

<ipython-input-37-6b6cdc35f513>:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=69, training_loss=0.10948425099469614, metrics={'train_runtime': 102.1658, 'train_samples_per_second': 10.806, 'train_steps_per_second': 0.675, 'total_flos': 290477213171712.0, 'train_loss': 0.10948425099469614, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.883857250213623,
 'eval_accuracy': 0.8064516129032258,
 'eval_precision': 0.8368565355624622,
 'eval_recall': 0.8064516129032258,
 'eval_f1': 0.8076372994603007,
 'eval_runtime': 2.9616,
 'eval_samples_per_second': 31.402,
 'eval_steps_per_second': 4.052,
 'epoch': 3.0}

In [ ]:
#save model
model.save_pretrained("./politicalbiasbert-finetuned")
tokenizer.save_pretrained("./politicalbiasbert-finetuned")

In [ ]:
from huggingface_hub import login
login()  # paste your HF token here

model.push_to_hub("vanikanoria/politicalbiasbert-finetuned")
tokenizer.push_to_hub("vanikanoria/politicalbiasbert-finetuned")

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vanikanoria/politicalbiasbert-finetuned/commit/b337880b2c0cedc7a65eaf3061e7308a38b72da8', commit_message='Upload tokenizer', commit_description='', oid='b337880b2c0cedc7a65eaf3061e7308a38b72da8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/vanikanoria/politicalbiasbert-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='vanikanoria/politicalbiasbert-finetuned'), pr_revision=None, pr_num=None)